# Preferential Bayesian Optimization: Multinomial Predictive Entropy Search

In [1]:
import numpy as np
import gpflow
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
import sys
import os
import pickle

print(tf.version.VERSION)

from gpflow.utilities import set_trainable, print_summary
gpflow.config.set_default_summary_fmt("notebook")

sys.path.append(os.path.split(os.path.split(os.path.split(os.getcwd())[0])[0])[0]) # Move 3 levels up directory to import project files as module
import importlib
PBO = importlib.import_module("Top-k-Ranking-Bayesian-Optimization")


2.6.0


In [2]:
gpu_to_use = 0

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        for gpu in gpus:
              tf.config.experimental.set_memory_growth(gpu, True)
        tf.config.experimental.set_visible_devices(gpus[gpu_to_use], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)

Num GPUs Available:  1
1 Physical GPUs, 1 Logical GPU


In [3]:
### Unique to real dataset ###

features = pickle.load( open( "sushi_features.p", "rb" ) ) # Initial data x (100x6)
#print(features.size)

fvals = pickle.load( open( "fvals.p", "rb" ) ) # Inital data y (1x100)
#print(fvals.size)

# construct dict
feat_to_fval_dict = {}
for i in range(len(features)):
    key = features[i].data.tobytes()
    feat_to_fval_dict[key] = fvals[i]

In [4]:
objective = lambda x: PBO.objectives.sushi(x, feat_to_fval_dict)
objective_low = np.min(features)
objective_high = np.max(features)
objective_name = "SUSHI"
acquisition_name = "MPES"
experiment_name = acquisition_name + "_" + objective_name

In [5]:
num_runs = 1 # number of experiments 
num_evals = 10 # number of queries
num_choices = 2 # 2 if pairwise
input_dims = 6
num_maximizers = 20 # maximizer = x* that maximizes the objective function 
num_maximizers_init = 50
num_fourier_features = 1000
num_init_prefs = 10 # number of initial observations

In [6]:
results_dir = os.getcwd() + '/results/' + experiment_name + '/'

try:
    # Create target Directory
    os.makedirs(results_dir)
    print("Directory " , results_dir ,  " created ") 
except FileExistsError:
    print("Directory " , results_dir ,  " already exists")

Directory  /home/johnson/Top-k-Ranking-Bayesian-Optimization/experiments/SUSHI/results/MPES_SUSHI/  already exists


In [7]:
def get_noisy_observation(X, objective): # noisy (human) observations
    f = PBO.objectives.objective_get_f_neg(X, objective)
    return PBO.observation_model.gen_observation_from_f(X, f, 1)

In [8]:
def train_and_visualize(X, y, title, lengthscale_init=None, signal_variance_init=None):
    
    # Train model with data
    result = PBO.models.learning_fullgp.train_model_fullcov(
                        X, y, 
                        obj_low=objective_low,
                        obj_high=objective_high,
                        lengthscale_init=lengthscale_init,
                        signal_variance_init=signal_variance_init,
                        indifference_threshold=0.,
                        n_sample=1000, # number of samples to estimate the probabilities in Eq. 8
                        deterministic=True, # only sample f values once, not re-sampling
                        num_steps=500) # how many optimization steps to take when training model
    
    q_mu = result['q_mu']
    q_sqrt = result['q_sqrt']
    u = result['u']
    inputs = result['inputs']
    k = result['kernel']
    
    likelihood = gpflow.likelihoods.Gaussian()
    model = PBO.models.learning.init_SVGP_fullcov(q_mu, q_sqrt, u, k, likelihood)
    u_mean = q_mu.numpy()
    inducing_vars = u.numpy()
    
    return model, inputs, u_mean, inducing_vars

Generate rank dictionary and immediate regret dictionary.

In [9]:
### Unique to real dataset ###

fval_idx_tuples = pickle.load(open("fval_idx_tuples.p", "rb"))

rank_dict = {}

for i in range(len(fval_idx_tuples)):
    rank_dict[features[fval_idx_tuples[i][1]].data.tobytes()] = i + 1

This function is our main metric for the performance of the acquisition function.

In [10]:
def get_max_sushi(model, features, rank_dict):
    """
    :param model: gpflow model
    :param features: sushi features
    :param rank_dict: dictionary from sushi idx to place in ranking
    :return: tuple (index of max sushi, rank)
    """
    f_preds = model.predict_f(features)[0]
    max_idx = np.argmax(f_preds)
    
    return (max_idx, rank_dict[features[max_idx].data.tobytes()])

Store the results in these arrays:

In [11]:
num_data_at_end = int(num_init_prefs + num_evals)
X_results = np.zeros([num_runs, num_data_at_end, num_choices, input_dims])
y_results = np.zeros([num_runs, num_data_at_end, 1, input_dims])
immediate_regret = np.zeros([num_runs, num_evals], np.int32)

Create the initial values for each run:

In [12]:
np.random.seed(0)

### Unique for real dataset ###
random_indices = np.random.choice(features.shape[0], [num_runs, num_init_prefs, num_choices])
init_vals = np.take(features, random_indices, axis=0)

The following loops carry out the Bayesian optimization algorithm over a number of runs, with a fixed number of evaluations per run.

In [13]:
for run in range(num_runs):  # CHECK IF STARTING RUN IS CORRECT
    print("Beginning run %s" % (run))
    
    X = init_vals[run]
    y = get_noisy_observation(X, objective)
    
    model, inputs, u_mean, inducing_vars = train_and_visualize(X, y, "Run_{}:_Initial_model".format(run))

    for evaluation in range(num_evals):
        print("Beginning evaluation %s" % (evaluation)) 
        
        success = False
        fail_count = 0
        while not success:
            # TODO: THIS ONLY WORKS FOR TOP-1 OF 2, CHANGE TO APPROPRIATE QUERY SAMPLING FOR HIGHER NUMBER OF CHOICES
            samples = PBO.models.learning_fullgp.construct_input_pairs(inputs, features)

            # Sample maximizers
            print("Evaluation %s: Sampling maximizers" % (evaluation))
            maximizers = PBO.fourier_features.sample_maximizers(X=inducing_vars,
                                                                count=num_maximizers,
                                                                n_init=num_maximizers_init,
                                                                D=num_fourier_features,
                                                                model=model,
                                                                min_val=objective_low,
                                                                max_val=objective_high,
                                                                num_steps=500) # Added by Leah Chong
            print(maximizers)

            # Calculate PES value I for each possible next query
            print("Evaluation %s: Calculating I" % (evaluation))
            I_vals = PBO.acquisitions.pes.I_batch(samples, maximizers, model)

            # Select query that maximizes I
            next_idx = np.argmax(I_vals)
            next_query = samples[next_idx]
            print("Evaluation %s: Next query is %s with I value of %s" % (evaluation, next_query, I_vals[next_idx]))

            X_temp = np.concatenate([X, [next_query]])
            # Evaluate objective function
            y_temp = np.concatenate([y, get_noisy_observation(np.expand_dims(next_query, axis=0), objective)], axis=0)
            
            try:
                print("Evaluation %s: Training model" % (evaluation))
                model, inputs, u_mean, inducing_vars = train_and_visualize(X_temp, y_temp,
                                                                           "Run_{}_Evaluation_{}".format(run, evaluation))
                success = True

            except ValueError as err:
                print(err)
                print("Retrying sampling random inputs")
                fail_count += 1

            if fail_count >= 3:
                print("Retry limit exceeded")
                raise ValueError("Failed")
                
        
        X = X_temp
        y = y_temp
        
        # Save model
        pickle.dump((X, y, inputs, 
                     model.kernel.variance, 
                     model.kernel.lengthscale, 
                     model.likelihood.variance, 
                     inducing_vars, 
                     model.q_mu, 
                     model.q_sqrt, 
                     maximizers), 
                    open(results_dir + "Model_Run_{}_Evaluation_{}.p".format(run, evaluation), "wb"))

        (max_idx, rank) = get_max_sushi(model, features, rank_dict)
        immediate_regret[run, evaluation] = rank - 1
        
        print("Maximizing sushi has index {} and rank {}".format(max_idx, rank)) 

    X_results[run] = X
    y_results[run] = y
    print("Run {} immediate regret: ".format(run))
    print(immediate_regret[run])

Beginning run 0
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.5 0.5 0.5 0.5 0.5 0.5]
       signal variance at 1.0
   Initial negative ELBO: 253.74041660531336
Negative ELBO at step 0: 251.5538582428323 in 0.6443s
Beginning evaluation 0
Evaluation 0: Sampling maximizers
Loss at step 0: -0.01831601459009835
Loss at step 500: -1.0622422575873083
Loss at step 1000: -1.33090823801621
Loss at step 1500: -1.3592598285443793
Loss at step 1695: -1.3598156381420585
test.shape =  (20, 50, 1)
tf.Tensor(
[[1.         0.00291997 0.57294442 0.35458208 0.40934695 0.59125691]
 [1.         0.         0.4393938  0.64922429 0.28895994 0.29204521]
 [0.96644831 0.         0.38832884 0.85110676 0.24258977 0.89259462]
 [0.90413062 0.         0.32121315 0.77562711 0.20299442 0.84045345]
 [0.97340432 0.         0.75644195 0.41654393 0.23086531 0.50

Evaluation 3: Next query is tf.Tensor(
[[1.         0.         0.49253731 0.14427861 0.57381325 0.04      ]
 [1.         0.         0.7706422  0.48929664 0.09563554 0.12      ]], shape=(2, 6), dtype=float64) with I value of 0.04859596125835299
Evaluation 3: Training model
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.5 0.5 0.5 0.5 0.5 0.5]
       signal variance at 1.0
   Initial negative ELBO: 646.3615518140523
Negative ELBO at step 0: 639.6848505922098 in 0.0811s
Maximizing sushi has index 57 and rank 27
Beginning evaluation 4
Evaluation 4: Sampling maximizers
Loss at step 0: 0.03529469227055927
Loss at step 500: -1.1396583801009406
Loss at step 1000: -1.4106328440394005
Loss at step 1500: -1.4405618346977085
Loss at step 2000: -1.4431345703995129
Loss at step 2467: -1.4445387525904974
test.shape =  (20, 50, 1)
tf.Tensor(

Evaluation 7: Next query is tf.Tensor(
[[1.         0.         0.78225806 0.24946237 0.38254217 0.08      ]
 [1.         0.         0.31390977 0.69172932 0.42259731 0.28      ]], shape=(2, 6), dtype=float64) with I value of 0.05555415474697835
Evaluation 7: Training model
Optimizer config:  {'name': 'RMSprop', 'learning_rate': 0.001, 'decay': 0.0, 'rho': 0.0, 'momentum': 0.0, 'epsilon': 1e-07, 'centered': False}
Indifference_threshold is fixed at 0.0
Initialize lengthscale at [0.5 0.5 0.5 0.5 0.5 0.5]
       signal variance at 1.0
   Initial negative ELBO: 1040.8386690044235
Negative ELBO at step 0: 1029.5955949952775 in 0.0982s
Maximizing sushi has index 79 and rank 9
Beginning evaluation 8
Evaluation 8: Sampling maximizers
Loss at step 0: 0.06453786165471825
Loss at step 500: -1.2103144158617962
Loss at step 1000: -1.4945385388100183
Loss at step 1500: -1.5316784081329007
Loss at step 2000: -1.5350214843662202
Loss at step 2500: -1.5363882036855643
Loss at step 2845: -1.5366250406454

In [14]:
pickle.dump((X_results, y_results, immediate_regret), open(results_dir + "res.p", "wb"))

In [15]:
ir = immediate_regret 
mean = np.mean(ir, axis=0)
std_dev = np.std(ir, axis=0)
std_err = std_dev / np.sqrt(ir.shape[0])

In [16]:
print("Mean immediate regret at each evaluation averaged across all runs:")
print(mean)

Mean immediate regret at each evaluation averaged across all runs:
[ 9.  8.  5. 26. 26.  8.  8.  8.  8.  8.]


In [17]:
print("Standard error of immediate regret at each evaluation averaged across all runs:")
print(std_err)

Standard error of immediate regret at each evaluation averaged across all runs:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [18]:
with open(results_dir + acquisition_name + "_" + objective_name + "_" + "mean_sem" + ".txt", "w") as text_file:
    print("Mean immediate regret at each evaluation averaged across all runs:", file=text_file)
    print(mean, file=text_file)
    print("Standard error of immediate regret at each evaluation averaged across all runs:", file=text_file)
    print(std_err, file=text_file)

In [19]:
pickle.dump((mean, std_err), open(results_dir + acquisition_name + "_" + objective_name + "_" + "mean_sem.p", "wb"))